In [1]:
import OAI
%load_ext autoreload 

In [2]:
h = OAI.Helper("demo_ask","./cache")
f = OAI.askFCT("demo_fct","./cache")

# Asking questions to any model, chat completion style

In [3]:
h.GOTOCACHE

'/home/kelu/llmcache/'

In [4]:
for overwrite in [True,False]:
    ans = h.ask("What is the size of the sun","answer with the size in km",v="gpt-3.5-turbo-1106",ow=overwrite)
    print(overwrite,"\t",ans)

Processing with gpt-3.5-turbo-1106 :	 11/22/2023, 09:16:55 --> 11/22/2023, 09:16:56
True 	 The diameter of the Sun is approximately 1.4 million kilometers.
False 	 The diameter of the Sun is approximately 1.4 million kilometers.


## Running with functions

In [5]:
tools = [
    {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        },
    {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }

]

In [6]:
CONTEXT = "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
Q = "What's the weather like today"

messages, chat_response = f.askFct(CONTEXT,Q,tools,modelGPT="gpt-3.5-turbo-1106",ow=True)
messages.append({"role": "user", "content": "The city is Dublin"})
msgs, answer = f.askFtcEngine(messages,tools,modelGPT="gpt-3.5-turbo-1106",ow=True)
f.pretty_print_conversation(msgs)


system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant: Sure, could you please specify the location for which you would like to know the weather?

user: The city is Dublin

assistant: Got it! And in what temperature unit would you like the weather information to be displayed? Celsius or Fahrenheit?



In [13]:
CONTEXT = "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
Q = "What's the weather like today"

messages, chat_response = f.askFct(CONTEXT,Q,tools,modelGPT="gpt-3.5-turbo-1106",ow=True)
messages.append({"role": "user", "content": "The city is Dublin"})
msgs, answer = f.askFtcEngine(messages,tools,modelGPT="gpt-3.5-turbo-1106",ow=True)
f.pretty_print_conversation(msgs)


system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant: Could you please specify the location for which you would like to know the weather?

user: The city is Dublin

assistant: {'name': 'get_current_weather', 'arguments': '{"location":"Dublin","format":"celsius"}'}



### Check for answers with direct answer

In [11]:
CONTEXT = "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
Q = "What's the weather like today in Dublin"

messages, chat_response = f.askFct(CONTEXT,Q,tools,modelGPT="gpt-3.5-turbo-1106",ow=True)
messages.append({"role": "user", "content": "The city is Dublin"})
msgs, answer = f.askFtcEngine(messages,tools,modelGPT="gpt-3.5-turbo-1106",ow=True)
f.pretty_print_conversation(msgs)


system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today in Dublin

assistant: {'name': 'get_current_weather', 'arguments': '{"location":"Dublin","format":"celsius"}'}

user: The city is Dublin

assistant: {'name': 'get_current_weather', 'arguments': '{"location":"Dublin","format":"celsius"}'}



In [12]:
messages[:-1]

[{'role': 'system',
  'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.\nDon't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
 {'role': 'user', 'content': "What's the weather like today in Dublin"},
 {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'get_current_weather',
   'arguments': '{"location":"Dublin","format":"celsius"}'}},
 {'role': 'user', 'content': 'The city is Dublin'}]